# **NVIDIA Stock Price Analysis**
## **Objective:**
- The primary objective of this project is to further my understanding of NVIDIA as a company and a stock market investment. This can be done using the following:
- Use Deep Learning & Neural Networks (application of research).
- Perform an EDA on NVIDIA's stock price, and relate this to the real world. This could involve:
    - Trend Analysis
    - Volatility Analysis
    - Earnings Reports
    - Balance Sheet Health
    - Sector Trends
    - Competitive Position
    - Sentiment of Other Investors
- Perform a predictive analysis on NVIDIA's stock price.
    - Using RNNs and LSTMs
- Perform the analysis above with reference to NVIDIA's competitors
    - Namely, AMD, Intel and QualComm
    

### Installations:

In [ ]:
%pip install pandas
%pip install yfinance

### Imports:

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf

import keras
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import os 

In [ ]:
keras.backend.set_image_data_format("channels_last")
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
yf.pdr_override()
%matplotlib inline

### Data Retreival & Cleaning:

In [ ]:
# Retrieve 5 Years of NVIDIA's Stock Data
stocks = ['NVDA', 'AMD', 'INTC', 'QCOM']
data_list = []

# now = datetime.now()
now = datetime(2023, 11, 1)


for stock in stocks:
    df = yf.download(stock, now - relativedelta(years=5), now)
    df['Stock'] = stock
    data_list.append(df)

c_names = ["NVIDIA", "AMD", "INTEL", "QUALCOMM"]
colors = {
    "NVIDIA": "green",
    "INTEL": "#6fa8dc", 
    "AMD": "red",
    "QUALCOMM": "darkblue" 
}

for c_data, c_name in zip(data_list, c_names):
    c_data['Company Name'] = c_name

data = pd.concat(data_list, axis = 0)

# Descriptive Statistics & Information
print(data.describe())
print(data.info())

# Data Cleaning

### EDA of Stock Price

#### Data Visualisation

In [ ]:
# Set up the matplotlib figure and axes for a 2x2 grid
fig, axs = plt.subplots(2, 2, figsize=(20, 10))
axs = axs.flatten()

for i, (c_data, c_name) in enumerate(zip(data_list, c_names)):
    c_data['MA50'] = c_data['Open'].rolling(50).mean()
    c_data['MA200'] = c_data['Open'].rolling(200).mean()
    
    # Direct Pandas plot to use a specific Matplotlib subplot
    c_data[['Open', 'MA50', 'MA200']].plot(ax=axs[i], figsize=(24, 15), title=f"{c_name} Stock Price and Moving Averages")
    
    axs[i].set_xlabel('Date')
    axs[i].set_ylabel('Price')
    axs[i].legend(["Open", "MA50", "MA200"])

plt.tight_layout()
plt.show()

In [ ]:
# Set the figure size and title
plt.figure(figsize=(24, 15))
plt.title("Stock Volumes")

# Filter and plot volume for each company
for c_name in c_names:
    # Filter the data for the current company
    company_data = data[data['Company Name'] == c_name]
    
    # Plot the volume data
    plt.plot(company_data.index, company_data['Volume'], label=c_name, color=colors[c_name], alpha=0.4, linewidth=2.5)

plt.legend(loc="best")
plt.grid(True)
plt.xlabel('Date')
plt.ylabel('Volume')
plt.show()

#### Annual Growth Rate

In [ ]:
# Calculating Compound Annual Growth Rate
cagr_results = {}
n_years = 5  # The period over which we're calculating CAGR

for c_data, c_name in zip(data_list, c_names):
    # Extract the closing price at the beginning and end of the period
    beginning_value = c_data['Adj Close'].iloc[0]
    ending_value = c_data['Adj Close'].iloc[-1]
    
    # Calculate CAGR
    cagr = ((ending_value / beginning_value) ** (1 / n_years)) - 1
    cagr_results[c_name] = cagr

# Print the CAGR for each company
for company, cagr in cagr_results.items():
    print(f"{company}: {cagr*100:.2f}% annual growth rate")

#### Correlation

In [ ]:
# Select only the 'Adj Close' column for each stock
adj_closes = pd.DataFrame({c_name: c_data['Adj Close'] for c_data, c_name in zip(data_list, c_names)})

# Correlation as PairPlot
tech_rets = adj_closes.pct_change()
sns.set(rc={'figure.figsize':(12,12)})
sns.set_context('talk')
sns.pairplot(tech_rets, kind='scatter', plot_kws={'alpha':0.1, 's':10})
sns.set_palette('deep')
sns.set_style('whitegrid', {'axes.grid' : True})

plt.show()

# Correlation as Heatmap
corr_matrix = tech_rets.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.show()

#### Risk VS Expected Return

In [ ]:
plt.figure(figsize=(24, 15))
area = np.pi * 20

# Iterate over each company name to calculate and plot their expected return and risk
for c_name in c_names:
    # Create a boolean mask for the current company
    mask = data['Company Name'] == c_name
    # Calculate the daily return and store it in a temporary variable instead of modifying the slice directly
    daily_return = data.loc[mask, 'Adj Close'].pct_change()
    # Now you can work with daily_return directly without modifying the original DataFrame
    mean_return = daily_return.mean()
    risk = daily_return.std()
    
    plt.scatter(mean_return, risk, s=area, label=c_name, alpha=0.9, linewidths=2, edgecolors='w')
    
    plt.annotate(f'{c_name} ({mean_return:.2%}, {risk:.2%})', 
        xy=(mean_return, risk), 
        xytext=(10, 5), 
        textcoords='offset points', 
        fontsize=12, 
        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))

plt.title('Expected Return vs. Risk for Each Company')
plt.xlabel('Expected Return')
plt.ylabel('Risk')

plt.legend(loc='upper right')
plt.grid(True)
plt.show()

#### Target Distribution

In [ ]:
# Create a new figure with a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(20, 20))

# Loop through each stock in your list and plot its target distribution
for i, (stock_data, stock_name) in enumerate(zip(data_list, c_names)):
    # Calculate the target variable (e.g., daily returns) for the current stock
    daily_return = stock_data['Adj Close'].pct_change()
    
    # Plot the target distribution using a KDE plot on the appropriate subplot
    row, col = divmod(i, 2)  # Calculate the row and column indices of the subplot
    sns.kdeplot(daily_return.dropna(), fill=True, ax=axs[row, col])
    axs[row, col].set_title(f'Target Distribution for {stock_name}', weight='bold', fontsize=20)
    axs[row, col].set_xlabel('Daily Return', fontsize=15)
    axs[row, col].set_ylabel('Density', fontsize=15)

# Adjust layout and spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# mean_target = daily_return.mean()
# median_target = daily_return.median()
# std_target = daily_return.std()
# skewness_target = daily_return.skew()
# kurtosis_target = daily_return.kurtosis()

# print(f"Mean: {mean_target}")
# print(f"Median: {median_target}")
# print(f"Standard Deviation: {std_target}")
# print(f"Skewness: {skewness_target}")
# print(f"Kurtosis: {kurtosis_target}")

In [ ]:
# Calculate statistics for each stock and print them
for stock_data, stock_name in zip(data_list, c_names):
    # Calculate target variable (e.g., daily returns) for the current stock
    daily_return = stock_data['Adj Close'].pct_change().dropna()
    
    # Compute statistics
    mean_target = daily_return.mean()
    median_target = daily_return.median()
    std_target = daily_return.std()
    skewness_target = daily_return.skew()
    kurtosis_target = daily_return.kurtosis()
    
    # Print the statistics for the current stock
    print(f"Target Distribution Statistics for {stock_name}:")
    print(f"Mean: {mean_target:.5f}")
    print(f"Median: {median_target:.5f}")
    print(f"Standard Deviation: {std_target:.5f}")
    print(f"Skewness: {skewness_target:.5f}")
    print(f"Kurtosis: {kurtosis_target:.5f}")
    print()

In [ ]:
# Create a new figure with a 2x2 grid of subplots
fig, axs = plt.subplots(2, 2, figsize=(20, 20))

# Define rolling windows for calculating the average target
rolling_windows = [10, 20, 50, 100]

# Loop through each stock in your list and plot its average target over different days
for i, (stock_data, stock_name) in enumerate(zip(data_list, c_names)):
    # Calculate the target variable (e.g., daily returns) for the current stock
    daily_return = stock_data['Adj Close'].pct_change()
    
    # Plot the average target over different days
    row, col = divmod(i, 2)  # Calculate the row and column indices of the subplot
    for window in rolling_windows:
        # Calculate the average target using a rolling window
        avg_target = daily_return.rolling(window).mean()
        
        # Plot the average target on the appropriate subplot
        axs[row, col].plot(avg_target, label=f'{window}-day Average')
        axs[row, col].set_title(f'Average Target over Different Days for {stock_name}', weight='bold', fontsize=20)
        axs[row, col].set_xlabel('Date', fontsize=15)
        axs[row, col].set_ylabel('Average Target', fontsize=15)
        axs[row, col].legend()

# Adjust layout and spacing
plt.tight_layout()

# Show the plot
plt.show()
